# hola
* adios
    
    * bueno
        * ahhh

Para crear el ambiente corremos en la terminal:
``` bash
conda create --name Ngram python=3.8.19
```

In [2]:
##year_1 and year_2 comprise the time period of ngrams to read##
year_1=2017
year_2=2018

import numpy
import scipy
import cython
import gensim
from gensim import models, similarities
import logging
import os
import re
import sys  
import itertools
import math


In [11]:
import os
import itertools
import gensim
import logging
import numpy

# Setup logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class MySentences(object):
    def __init__(self, dirname, start_year, end_year, limit=None):
        self.dirname = dirname
        self.start_year = start_year
        self.end_year = end_year
        self.limit = limit

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            with gensim.utils.open(os.path.join(self.dirname, fname)) as fin:
                for line in itertools.islice(fin, self.limit):
                    line = gensim.utils.to_unicode(line).split("\t")
                    if len(line) < 3:
                        continue
                    ngram = line[0]
                    try:
                        year = int(line[1])
                    except ValueError:
                        continue
                    match_count = int(line[2])
                    if year < self.start_year or year > self.end_year:
                        continue
                    processed_ngram = [word.split("_")[0] for word in ngram.lower().split()]
                    for x in range(match_count):
                        yield processed_ngram

# Define year_1 and year_2
year_1 = 2000
year_2 = 2010

# Create sentences iterator
sentences = MySentences("C:/Users/malfa/Downloads/ngrams/ngrams2", year_1, year_2)

# Build the vocabulary
model = gensim.models.word2vec.Word2Vec(vector_size=300, window=5, min_count=10, workers=10, hs=0, negative=8)
model.build_vocab(sentences)

# Train word2vec model
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

# Save the model
model.save('w2vmodel_ng5_'+str(year_1)+'_'+str(year_2)+'_full')

# Save the vectors
numpy.savetxt('syn0_ngf_'+str(year_1)+'_'+str(year_2)+'_full.txt', model.wv.vectors, delimiter=" ")

# Save the vocabulary list
vocab_list = model.wv.index_to_key
for i in range(0, len(vocab_list)):
    if vocab_list[i] == '':
        vocab_list[i] = "thisisanemptytoken"+str(i)

with open('vocab_list_ngf_'+str(year_1)+'_'+str(year_2)+'_full.txt', 'wb') as outfile:
    for i in range(0, len(vocab_list)):
        outfile.write(vocab_list[i].encode('utf8')+"\n".encode('ascii'))


2024-04-16 18:34:01,835 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.025>', 'datetime': '2024-04-16T18:34:01.835127', 'gensim': '4.3.2', 'python': '3.8.19 | packaged by conda-forge | (default, Mar 20 2024, 12:38:07) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2024-04-16 18:34:01,835 : INFO : collecting all words and their counts
2024-04-16 18:34:01,839 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-04-16 18:34:01,856 : INFO : PROGRESS: at sentence #10000, processed 50000 words, keeping 524 word types
2024-04-16 18:34:01,873 : INFO : PROGRESS: at sentence #20000, processed 100000 words, keeping 873 word types
2024-04-16 18:34:01,891 : INFO : PROGRESS: at sentence #30000, processed 150000 words, keeping 1087 word types
2024-04-16 18:34:01,906 : INFO : PROGRESS: at sentence #40000, processed 200000 words, keeping 1330 word types
2024-04-16 18:34:01,918 : INFO : PROGR

In [ ]:
import os
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from collections import Counter
import itertools

# Placeholder for the data directory and years
dirname = "/ngrams/googlebooks-eng-all-5gram-20120701-0/googlebooks-eng-all-5gram-20120701-0/"
start_year = 2000
end_year = 2012

# Function to read and preprocess n-grams
def read_ngrams(dirname, start_year, end_year, limit=None):
    vocab = Counter()
    data = []
    for fname in os.listdir(dirname):
        with open(os.path.join(dirname, fname), 'r') as fin:
            for line in itertools.islice(fin, limit):
                parts = line.strip().split("\t")
                if len(parts) < 3:
                    continue
                ngram, year, match_count = parts[0], int(parts[1]), int(parts[2])
                if start_year <= year <= end_year:
                    words = [word.split("_")[0].lower() for word in ngram.split()]
                    vocab.update(words)
                    data.append(words)
    return data, vocab

data, vocab = read_ngrams(dirname, start_year, end_year)

# Function to build a vocabulary index
def build_vocab_index(vocab):
    vocab_index = {word: i for i, (word, _) in enumerate(vocab.most_common(), 1)}
    return vocab_index

vocab_index = build_vocab_index(vocab)

# Function to convert words to indices
def words_to_indices(data, vocab_index):
    indexed_data = []
    for sentence in data:
        indexed_sentence = [vocab_index[word] for word in sentence if word in vocab_index]
        indexed_data.append(indexed_sentence)
    return indexed_data

indexed_data = words_to_indices(data, vocab_index)
